# NeutrinoFlux
This package predicts the yearly rate of neutrino events in the IceCube Neutrino Observatory at the south pole. For this purpose, this package also includes modules with functions for neutrino cross sections, differential flux from atmospheric and astrophysical sources, and attenuation through the earth.

To allow for compatibility with other packages, this package uses classes for `Neutrino`s and `CrossSection`s, whose default attributes and methods can be replaced with other functions as desired.

For a full report on the mathematics used in this package, see `NeutrinoFlux.pdf`.

In [1]:
import NeutrinoFlux
ureg = NeutrinoFlux.ureg

MCEqRun::set_interaction_model(): SIBYLL23C
ParticleManager::_init_default_tracking(): Initializing default tracking categories (pi, K, mu)
MCEqRun::set_density_model(): Setting density profile to MSIS00_IC ('SouthPole', 'January')
MSIS00IceCubeCentered::set_theta(): latitude = -90.00 for zenith angle =  0.00
MCEqRun::set_primary_model(): HillasGaisser2012 H3a


## Usage
As a quick example, the function call below computes the yearly rate of electron neutrinos from astrophysical sources, from all zenith angles, for energies between $10^{4}$ and $10^{12}$ GeV.

In [2]:
NeutrinoFlux.event_rate(
    nu=NeutrinoFlux.nu["nu_e"],
    flux_type="astro"
)

<Quantity(118.55463374803169, '1 / year')>

If we wanted to compute only the rate of Glashow Resonance events for electron antineutrinos within the same range, but with a spectral index of $\gamma = 2$ (instead of the default $2.53$) and a normalization of $\phi_{\text{astro}} = 1$ (instead of the default $1.66$), I would instead write:

In [3]:
nubar_e = NeutrinoFlux.nu["nubar_e"]
NeutrinoFlux.event_rate(
    nu=nubar_e,
    flux_type="astro",
    cross_sections=nubar_e.cross_sections["gr"],
    gamma=2,
    phi_astro=1
)

<Quantity(5.745772089556312, '1 / year')>

If we instead consider atmospheric mu neutrino flux from pion decay in July with energies only up to $10^{16}$ eV, we would write

In [18]:
NeutrinoFlux.event_rate(
    nu=NeutrinoFlux.nu["nu_mu"],
    flux_type="atmo",
    E_bounds=(1e13, 1e16) * ureg.eV,
    month="July",
    source="pi",
    N = 500
).to("1/month")

<Quantity(7.314061187019767, '1 / month')>

## Compatibility
While this package provides a set of models by default, other models can be incorporated by modifying a `CrossSection` or `Neutrino` object's attributes.

For example, the [`nuFATE`](https://github.com/aaronvincent/nuFATE) can be used to compute the attenuation of initial flux for a given flavor, including $\nu_{\tau}$ regeneration effects. To use this alongside `NeutrinoFlux`, all it would take is taking a `Neutrino`-class object and replacing its default `attenuation` method with one that uses the `nuFATE` attenuation model. This custom `Neutrino` can then be passed into the `event_rate` function as normal.